## Demo of fmm Python API

In [1]:
from fmm import Network,NetworkGraph,FastMapMatch,FastMapMatchConfig,UBODT
from fmm import UBODTGenAlgorithm
#! pip2 install pyarrow
import pyarrow
import pyarrow.parquet as pq
import os

In [2]:

def getPoints(file_path):
    if os.path.isfile(file_path):
        text_file = open(file_path, "r")
        wkt = text_file.read()
        text_file.close()
    return wkt
def save_results(file_base_name, cpath, opath, indices, m_wkt, e_wkt):
  text_file = open(file_base_name + '-cpath.txt', "w")
  text_file.write(cpath)
  text_file.close()
  text_file = open(file_base_name + '-opath.txt', "w")
  text_file.write(opath)
  text_file.close()
  text_file = open(file_base_name + '-indices.txt', "w")
  text_file.write(indices)
  text_file.close()
  text_file = open(file_base_name + '-m_wkt.txt', "w")
  text_file.write(m_wkt)
  text_file.close()
  text_file = open(file_base_name + '-e_wkt.txt', "w")
  text_file.write(e_wkt)
  text_file.close()

### Load network data and graph

In [ ]:
#network = Network("data/edges.shp")
network = Network('data/GPS.nosync/street_nodes_dual.shp')
print "Nodes {} edges {}".format(network.get_node_count(),network.get_edge_count())
print(network)
graph = NetworkGraph(network)

### Precompute an UBODT file

**This step can be skipped if you already created one UBODT file.**

In [4]:
ubodt_gen = UBODTGenAlgorithm(network,graph)

In [ ]:
status = ubodt_gen.generate_ubodt("data/GPS.nosync/street_dual_ubodt_new.txt", 4, binary=False, use_omp=True)
print status

### Load UBODT data

In [7]:
ubodt = UBODT.read_ubodt_csv("data/GPS.nosync/street_dual_ubodt_new.txt")

### Create FMM model

In [8]:
model = FastMapMatch(network,graph,ubodt)

### Define FMM configuration

In [74]:
# k = 50
# radius = 100
# gps_error = 50
k = 64
radius = 10
gps_error = 50
fmm_config = FastMapMatchConfig(k,radius,gps_error)

### Run map matching

#### Match a single trajectory

In [10]:
#wkt = "LINESTRING(0.200812146892656 2.14088983050848,1.44262005649717 2.14879943502825,3.06408898305084 2.16066384180791,3.06408898305084 2.7103813559322,3.70872175141242 2.97930790960452,4.11606638418078 2.62337570621469)"

In [96]:
def get_sub_result(input_path, output_path, index):
  input_source = input_path + str(index) + '-points.txt'
  out_dir = output_path + 'sub-result-' + str(index)
  if (os.path.isfile(input_source)) & (not os.path.isdir(out_dir)):
    wkt = getPoints(input_source)
    result = model.match_wkt(wkt,fmm_config)
    if len(list(result.cpath)) > 0:
      #dir = output_path + 'sub-result-' + str(index)
      os.mkdir(out_dir)
      output_base = out_dir + '/' + str(index)
      save_results(output_base,str(list(result.cpath)), str(list(result.opath)),str(list(result.indices)),result.mgeom.export_wkt(),result.pgeom.export_wkt())
      print('index %d sucess!' % index)
    else:
      print('index %d failed!' % index)



In [98]:
#get_sub_result('data/GPS.nosync/points/', 'data/GPS.nosync/output/', 4)

In [ ]:
for index in range(5,999):
  get_sub_result('data/GPS.nosync/points/', 'data/GPS.nosync/output/', index)

In [82]:
wkt = getPoints('data/GPS.nosync/points/4-points.txt')
#wkt = getPoints('data/GPS.nosync/output/00074014459977889ff22e04d309cbc68fb1720990d2d6a7251eb6c17fbf5f72-points.txt')

In [ ]:
result = model.match_wkt(wkt,fmm_config)
print "Matched path: ", list(result.cpath)
# print "Matched edge for each point: ", list(result.opath)
# print "Matched edge index ",list(result.indices)
# print "Matched geometry: ",result.mgeom.export_wkt()
# print "Matched point ", result.pgeom.export_wkt()

In [ ]:
len(list(result.cpath)) > 1

In [ ]:
save_results('data/GPS.nosync/output/0-00074',str(list(result.cpath)), str(list(result.opath)),str(list(result.indices)),result.mgeom.export_wkt(),result.pgeom.export_wkt())

##Print detailed match information

In [ ]:
candidates = []
for c in result.candidates:
    candidates.append((c.edge_id,c.source,c.target,c.error,c.length,c.offset,c.spdist,c.ep,c.tp))

In [ ]:
import pandas as pd
df = pd.DataFrame(candidates,
                  columns=["eid","source","target","error","length","offset","spdist","ep","tp"])

In [ ]:
df.head()

#### Match trajectories in a GPS file

In [ ]:
from fmm import GPSConfig,ResultConfig

In [ ]:
input_config = GPSConfig()
input_config.file = "../data/trips.csv"
input_config.id = "id"

In [ ]:
print input_config.to_string()

In [ ]:
result_config = ResultConfig()
result_config.file = "../data/mr.txt"
result_config.output_config.write_opath = True
print result_config.to_string()

In [ ]:
status = model.match_gps_file(input_config, result_config, fmm_config)

In [ ]:
print status

In [ ]:


filename = "../data/in_zone_grouped_evacuation_stops.parquet" 



df = pq.read_table(source=filename).to_pandas()

In [ ]:
print(df.loc[100])